<a href="https://colab.research.google.com/github/NatalyaEvans/ETNP_stoich/blob/main/pyompa/endmember_sensitivity_fns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is designed to initialize functions to determine sensitivities in endmember nutrients

Make the stoichiometry perturbation more robust
Comment more
Make sure that the full pyompa script can load this in

First, load in weightings sensitivity

In [ ]:
#Install gp15wmascripts
!pip uninstall -y gp15wma
%cd /content/
!rm -rf gp15wmascripts
!git clone https://github.com/nitrogenlab/gp15wmascripts
%cd /content/gp15wmascripts
!git checkout main
!git log -1
!pip install .
%cd /content/

%reload_ext autoreload
%autoreload 2
from importlib import reload
import gp15wma
reload(gp15wma)

In [ ]:
from gp15wma.sensitivity import BaseSensitivityAnalysis, OmpaArguments

Next, define functions for endmember sensitivities using the weightings sensitivity structure

In [ ]:
def perturb_endmember_df(df, variation_range, rng):
  new_df = OrderedDict([
      ("Params", df["Params"]) #endmember name column
  ])
  params_to_perturb = ["CT", "SA", "Phosphate", "Nitrate", "Silicate", "tCO2"]
  for param_name in params_to_perturb:
    values = list(df[param_name])
    new_values = [rng.uniform(value-variation_range[param_name],
                              value+variation_range[param_name])
                  for value in values]
    new_df[param_name] = new_values

  return pd.DataFrame(new_df)


def perturb_endmember_df_stoichiometrically(df, stoichiometry, rng,
                                            perturbation_limits=(-1,1)):
  #in Talia's project, pertubation_limits corresponds to phosphate which
  # has ratio 1 wrt to the converted param
  
  new_df = OrderedDict([
      ("Params", df["Params"]), #endmember name column
      ("CT", df["CT"]),
      ("SA", df["SA"]),
  ])

  params_to_perturb = ["Phosphate", "Nitrate", "Silicate", "tCO2"]
  #sample a different perturbation for each row in the data frame
  sampled_perturbations = [rng.uniform(perturbation_limits[0],
                                       perturbation_limits[1])
                           for i in range(len(df))]
  print(sampled_perturbations)
  for param_name in params_to_perturb:
    values = list(df[param_name])
    new_values = [value + sampled_perturbation*stoichiometry[param_name]
                  for sampled_perturbation,value in
                  zip(sampled_perturbations,values)]
    new_df[param_name] = new_values

  return pd.DataFrame(new_df)

variation_range = {
    "CT": 3, "SA": 0.75, "Phosphate": 1.5, "Nitrate": 10,
    "Silicate": 20, "tCO2": 100
}

perturb_endmember_df_stoichiometrically(
    endmembers_df,
    stoichiometry=convertedparamgroups[0].conversion_ratios[1],
    perturbation_limits=(-0.5, 0.5),
    rng = np.random.RandomState(1234))